In [1]:
!pip install implicit==0.4.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 26.1 MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3423115 sha256=f2f1f7aa23494db470a63f341c67dcb77f354af11852a6552f09a23945543eb5
  Stored in directory: /root/.cache/pip/wheels/44/7e/7d/a17324ea207cfbe76aca878b5b8ca0aa932cf55d163329be37
Successfully built implicit


In [2]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  PATH = '/content/drive/MyDrive/Move_to/GEEK_brain/Рекомендательные системы/webinar_3/'

else:

  PATH = 'D:/'

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

from tqdm import tqdm

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)


try:    
  from metrics import precision_at_k, recall_at_k

except:
  sys.path.insert(0,PATH)
  from metrics import precision_at_k, recall_at_k

In [4]:
try:
  data = pd.read_csv('../data/transaction_data.csv')
except:
  data = pd.read_csv(PATH + 'data/transaction_data.csv')

data.head(2)

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [6]:
# df_popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
# item_features = pd.read_csv('../data/product.csv')
# item_features.columns = [col.lower() for col in item_features.columns]
# item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

# item_features.head(2)
# item_features.department.unique()

In [7]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [8]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [9]:
data_train.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [10]:
# Заведем фиктивный item_id

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999_999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(3)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [12]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

## Задание 1. 
Поэкспериментировать с ALS (grid-search) ****


In [63]:
# Взвесим с bm25_weight
user_item_matrix = bm25_weight(user_item_matrix.T).T

Определим гиперпараметры для перебора

In [59]:
factors = [8, 16, 32, 64, 128]
regularization = [0.01, 0.05, 0.1]

In [60]:
%%time

metric_value = {}

for factor in tqdm(factors):
    for reg in regularization:
        # for itr in iterations:
          model = AlternatingLeastSquares(factors=factor, 
                                          regularization=reg,
                                          iterations=10, 
                                          calculate_training_loss=True, 
                                          num_threads=10,
                                          use_gpu=True)

          model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
                    show_progress=False)
          
          result[f'{factor}_{reg}'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

          metric_value[f'{factor}_{reg}'] = np.nan
          metric_value[f'{factor}_{reg}'] = result.apply(lambda row: precision_at_k(row[f'{factor}_{reg}'], row['actual']), axis=1).mean()


100%|██████████| 5/5 [05:43<00:00, 68.60s/it]

CPU times: user 6min 3s, sys: 4min 43s, total: 10min 47s
Wall time: 5min 43s


In [61]:
import plotly.express as px
    

columns=['factors', 'regularization', 'precision_at_k']    
xyz = pd.DataFrame([[i.split('_')[0] for i in list(metric_value.keys())], [i.split('_')[1] for i in list(metric_value.keys())], metric_value.values()], index = columns).T

fig = px.scatter_3d(xyz, 
                    x=columns[0], 
                    y=columns[1], 
                    z=columns[2], 
                    title= 'Зависимость precision_at_k от factors и regularization',
                    width=1000,
                    height=1000)
fig.show()

In [62]:
metric_value

{'128_0.01': 0.19065796082370406,
 '128_0.05': 0.18895027624309135,
 '128_0.1': 0.190055248618782,
 '16_0.01': 0.1586137619286771,
 '16_0.05': 0.16233048719236337,
 '16_0.1': 0.15700652938221782,
 '32_0.01': 0.15781014565544757,
 '32_0.05': 0.15981918633852127,
 '32_0.1': 0.15931692616775267,
 '64_0.01': 0.16966348568558287,
 '64_0.05': 0.17388247112003785,
 '64_0.1': 0.17307885484680813,
 '8_0.01': 0.15931692616775253,
 '8_0.05': 0.1580110497237549,
 '8_0.1': 0.15921647413359938}

Ожидаемо лучше при максимальном числе factor. Регуляризация не принципиальна

## Задание 2.
Ситуация: Вы работаете data scientist в крупном продуктовом российском ритейлере. Ваш конкурент сделал рекомендательную систему, и его продажи выросли. Ваш менеджмент тоже хочет увеличить продажи.


- А точно нужно сортировать по вероятности?

1. Возможно нужно предварительно сгруппировать товары и брать лучшие по вероятности из каждой группы (потому что никому не интересно поулчать топ самых вероятных, которые принадлежат одному типу товара). Эти группировке по цене, по качеству, стране производителю итп. 
2. Можно учесть рентабельность (брать проивзедение вероятности на рентабельность).

- Какую метрику использовать?

1. Опять же если нас интересует финансовый результат, то простое угадывание товаров малоинтересно. Интересно максимизировать рентабельность. 

- Сколько раз в неделю отпрпавляем рассылку?

1. Провести маркетинговое исследование на вопрос при какой частоте рассылок клиенты отказываются от  сотрудничество. 
2. Если рассылки платные то можно посчитать при какой частоте достигается максимальный экономический эффект

- В какое время отправляем рассылку?

1. минус 1 час от времени когда люди обычно совершают покупки (19-20 часов)
2. отслеживать людей (яндекс такси, GPRS) и кидать рассылку в момент направления в магазин (ну это шутка)
3. В момент списания средств в продуктовых магазинах, кидать рассылку: "Доп товары которые вы могли забыть купить"


- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?

1. Составить базу однотипных товаров (имен, обращений) и случайно выбирать из списка
2. Использовать NLP которая будет менять стилистику (молодежный сленг, сленг для пенсионеров) предложение
3. Натренировать несколько моделей, с примерно одинаковой мтерикой но с разным набором предложений. Кидать рассылку предложений из разных моделей.
4. Учитывать сезонность, дни недели, время суток итп итд 
5. Рандомно выбирать 5 из топ 15.
6. Учиться на разных наборах данных (за разные года, от разных магазинов, стран )

- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?

1. Нужно, люди не любят ботов. Разные - после кластеризации попадающие в разные кластеры, имеющие разные меры сходства (косинусное к примеру), которые не пересекаются у одого и того же покупател (группы схожих покупателей). Как добиться - частично ответил в предыдущем пункте. Можно делать группировки по брендам, ценам, скидкам и по каждой отдельно оубчать модель. Или разбить пользоваталей по размеру чека и по каждой группе делать свою модель, а затем предлагать пользователю товары из модели обученной на чуть более тратящихся покупателей. Или наоборот, назвав рассылку - "Рекомендации однотипных товаров, но за меньшие деньги".



# ДЗ

## Теория:

1) Попытаться ответить на вопросы/выдвинуть гипотезы

2) Доделать прошлые домашния задания

3) Прочитать статьи BM25/MatrixFactorization

## Практика:

4) Поэкспериментировать с ALS (grid-search) ********
   * Подобрать лучшие параметры
   * Написать отчет об увиденном

# Links

BM25

https://en.wikipedia.org/wiki/Okapi_BM25#:~:text=BM25%20is%20a%20bag%2Dof,slightly%20different%20components%20and%20parameters.


Matrix factorization (ALS, SVD)
https://datasciencemadesimpler.wordpress.com/tag/alternating-least-squares/